## Evaluating the output

In [ ]:
import numpy as np 
import pandas as pd
from scipy.stats import ttest_ind
from scipy.stats import ttest_1samp, binom

import json 
import copy
import jsonlines
import os
import openai
from openai import AzureOpenAI
from tqdm import tqdm

In [ ]:
endpoint = ""
api_key = "" 

openai.azure_endpoint = endpoint  # your endpoint should look like "https://<your-resource-name>.openai.azure.com/"  
openai.api_version = ""  # specify the API version you're using  
openai.api_key = api_key  
openai.api_type = "azure"  

def call_llm(query):

    completion = openai.chat.completions.create(
        model="gpt-4o",
        messages=[        
            {
                "role": "user",
                "content": query
            }
        ],
        temperature=0.0,
        #response_format={"type": "json_object"}
    )
        
    return completion.choices[0].message.content

def check_table(text, prompt = None):

    if prompt is None:
        prompt = """
    You are to check if there is any table in the text given. 
    Lists and equations are not tables so do not include them. 

    TEXT: {text}

    #OUTPUT 
    You are to give your answer in JSON format which is either
    {{"Table": "True"}} or {{"Table": "False"}}
    """

    try:
        response = call_llm(prompt.format(text = text))
    except Exception as e:
        print(f"Error calling LLM: {e}")
        response = None             

    return eval(response)

def check_tone(text_1, text_2 , prompt = None):

    if prompt is None:
        prompt = """
    You are given 2 texts. You are to judge which of them comes from an financial expert based on the tone. 
    You are also to give the reason why you think it is so.
    TEXT 1: {text_1}

    TEXT 2: {text_2}

    #OUTPUT 
    You are to give your answer in JSON format which is either
    {{"Judgement": "TEXT_1",
      "Reason"   : "..."}} 
    or
    {{"Judgement": "TEXT_2",
      "Reason"   : "..."}}
    """

    try:
        response = call_llm(prompt.format(text_1, text_2))
    except Exception as e:
        print(f"Error calling LLM: {e}")
        response = None               

    return eval(response)

In [2]:
Finetune_answer_example = \
''' 
For the period ended 31 March 2022, the company reported other income of AED 252 million. The main contributor to this income was dividend income from the company's investment in the company Limited 

| Description | Amount (AED million) |
|-------------|----------------------|
| Other Income | 252 |
| Main Contributor | Dividend Income  |
'''

Control_answer_example = \
''' 
For the period ended 31 March 2022, the company reported other income of AED 252 million. The main contributor to this other income was dividend income from the company's investment in the company Limited.

'''

In [ ]:
print(check_table(Finetune_answer_example))
print(check_tone(Finetune_answer_example, Control_answer_example))